# **4조**
**오차역전파(계산 그래프, 연쇄법칙, 역전파)**

## 1. 계산 그래프 및 연쇄법칙 (15점)

문제 1 : Local Gradient, Global Gradient가 무엇인지 서술하세요. (5점)

정답 : Local Gradient는 모델의 특정 파라미터에 대한 손실 함수의 기울기를 의미한다. 
Global Gradient는 모델의 모든 파라미터에 대한 손실 함수의 기울기를 통합하여 계산한 값이다.

문제 2 : 역전파 과정에서는 <>를 활용하여 <>, <>를 곱해 최종 기울기를 계산합니다. <>에 들어갈 단어를 순서대로 적어주세요. (5점)

정답 : 체인 룰 , 국소 기울기

문제 3 : 계산 그래프에서 각각의 노드가 순전파와 역전파에서 어떠한 역할을 하는지 전반적인 흐름을 서술하세요. (5점)

정답 : 순전파에선 입력 데이터를 입력 노드로 전달 후에 각 연산 노드는 입력을 처리하여 결과를 계산한다. 그 결과를 다음 노드로 전달하여 마지막에 최종 출력을 계산한다.
역전파에선 출력 노드에서 손실 함수의 기울기를 계산한다.
각 연산 노드는 체인 룰을 이용해 기울기를 계산 후 이를 이전 노드로 전달한다.
기울기는 네트워크를 거꾸로 따라 전파되어, 각 노드가 기울기를 계산하고
이를 통해 파라미터를 업데이트할 준비를 한ㄷ.

## 2. 역전파 (35점)

문제 1 : 역전파 알고리즘의 작동 방식은 손실함수의 오차값 (<> - <>)을 계산하여 왼쪽으로 전파해가며 각 노드가 갖고 있는 <> 값을 갱신해가는 과정입니다. <>에 들어갈 단어를 순서대로 적어주세요. (5점)

정답 : 예측값, 실제값, 기울기

문제 2 : 하단의 코드는 간단한 계산 그래프를 구축하여 순전파와 역전파를 구현하는 클래스들이며, 각각의 클래스는 곱셈과 덧셈을 수행합니다. 계산 그래프의 동작 방식을 살펴보기 위해 첫 번째 셀의 코드를 채우고, 두 번째 셀의 코드를 실행해주세요. (10점)

In [2]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y  
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x 
        return dx, dy

class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y  
        return out
    
    def backward(self, dout):
        dx = dout 
        dy = dout  
        return dx, dy


In [3]:
# 코드 수정없이 실행

# 입력 값 설정
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층 생성
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
total_price = add_apple_orange_layer.forward(apple_price, orange_price)
price_with_tax = mul_tax_layer.forward(total_price, tax)

# 역전파
dprice_with_tax = 1
dtotal_price, dtax = mul_tax_layer.backward(dprice_with_tax)
dapple_price, dorange_price = add_apple_orange_layer.backward(dtotal_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(f"사과 가격에 대한 미분: {dapple:.1f}, 사과 개수에 대한 미분: {dapple_num:.0f}")
print(f"귤 가격에 대한 미분: {dorange:.1f}, 귤 개수에 대한 미분: {dorange_num:.0f}")
print(f"소비세에 대한 미분: {dtax:.0f}")

사과 가격에 대한 미분: 2.2, 사과 개수에 대한 미분: 110
귤 가격에 대한 미분: 3.3, 귤 개수에 대한 미분: 165
소비세에 대한 미분: 650


문제 3 : 세션 때 배운 내용을 토대로 세 번째 셀의 코드를 채워주세요. (10점)

In [ ]:
# 라이브러리가 없을 경우 실행

!pip install torch torchvision

In [4]:
# 코드 수정없이 실행

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

input_size = 10
num_classes = 2
batch_size = 4
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 임의의 데이터와 레이블 생성
data = torch.randn(100, input_size)
labels = torch.randint(0, num_classes, (100,))
dataset = TensorDataset(data, labels)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 간단한 RNN 모델 정의
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), hidden_size).to(device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

hidden_size = 20
rnn_model = SimpleRNN(input_size, hidden_size, num_classes).to(device)

# 손실 함수와 최적화 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn_model.parameters(), lr=0.001)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\LWG\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\LWG\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\LWG\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.sta

In [5]:
# 모델 학습
for epoch in range(num_epochs):
    rnn_model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        # 입력 데이터를 3D 형태로 변환 (batch_size, seq_len, input_size)
        images = images.unsqueeze(1) 
        # 기울기 초기화
        optimizer.zero_grad()
        # 순전파  
        outputs = rnn_model(images)
        # 손실 계산  
        loss = criterion(outputs, labels)
        # 역전파  
        loss.backward()
        # 최적화  
        optimizer.step()  
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

Epoch 1, Loss: 0.7077111601829529
Epoch 2, Loss: 0.698150520324707
Epoch 3, Loss: 0.690264413356781
Epoch 4, Loss: 0.6834771823883057
Epoch 5, Loss: 0.680760223865509


문제 4 : 지금까지의 세션에서의 배운 내용을 토대로 Gradient와 Jacobian의 사용 예시를 각각 서술해주세요. (10점)

정답 : Gradient는 경량화된 Gradient Descent에서 사용할 수 있고, 
Jacobian은 다변량 함수의 민감도 분석에서 사용할 수 있다.

# **5조**

# 1. 활성화 함수 이론 및 구현 코드에 대한 문제입니다.(25점)

### 1-1. Binary Step Function, Linear Activation Function, Non-Linear Activation Function의 개념과 Non-Linear Activation Function를 사용하는 이유를 간단하게 작성해주세요.(3점)

답)

Binary Step Function: Binary Step Function은 입력이 특정 임계값(threshold)을 초과하면 1을 반환하고, 그렇지 않으면 0을 반환하는 함수이다.

Linear Activation Function:  Linear Activation Function은 입력을 그대로 출력으로 반환하는 함수이다.

Non-Linear Activation Function: Non-Linear Activation Function은 입력에 비선형 변환을 적용하여 출력하는 함수입니다. 예를 들어, ReLU(Rectified Linear Unit), Sigmoid, Tanh 등이 있다.

Non-Linear Activation Function를 사용하는 이유 : 비선형 활성화 함수는 네트워크가 입력과 출력 사이의 비선형 관계를 학습할 수 있게 하고, 이는 단순 선형 함수로는 표현할 수 없는 복잡한 패턴을 모델링할 수 있게 해준다.

### 1-2. ???를 작성해주세요. (3점)

sigmoid 함수 : 출력이 0과 1사이로 나와 층을 깊이 쌓으면 gradient vanising 문제가 있고 computational cost가 높습니다.


Tanh 함수 : [-1, 1]범위로 shift된 형태를 띄며, 층이 쌓일수록 gradient 값이 0에 수렴하여 학습 효율이 낮아진다는 특징이 있습니다.


ReLU 함수 : 현재 hidden layer의 activation function로 가장 많이 사용되며 간단하고 계산이 효율적인 함수입니다.

### 1-3. Relu 계층을 구현한 코드입니다. 코드를 완성해주세요.(5점)

In [7]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx


### 1-4. 위 코드를 보고 forward 메서드에서 self.mask의 역할을 설명하고, 이를 통해 어떻게 ReLU 활성화 함수가 작동하는지 설명하세요.(3점)

답) 입력값 x의 각 요소가 0 이하인지를 나타내는 불리언 배열이다. (x <= 0) 코드에서 self.mask는 x의 요소 중 0 이하인 위치에 True, 0보다 큰 위치에 False 값을 갖게 된다. 이를 통해 입력값이 0 이하였던 위치에서는 gradient가 전달되지 않게 되고, 나머지 위치에서는 gradient가 그대로 전달된다. 이는 ReLU의 미분 특성을 반영한 것이다.

### 1-5. 위 코드를 통해 backward 메서드에서 dout[self.mask] = 0 코드는 어떤 역할을 하는지 설명하세요.(3점)

답) dout[self.mask] = 0 코드는 ReLU 함수의 미분 특성을 반영하여, 입력값이 0 이하인 위치의 gradient를 0으로 만든다.
이를 통해 역전파 과정에서 해당 위치의 gradient가 전파되지 않도록 한다.

### 1-6. 아래는 PyTorch 기반 DenseBlock로 표현한 Relu 함수 코드입니다. Relu 함수에서는 Parameters로 'inplace'가 사용될 수 있는데, 'inplace = True'일 때 특징을 간단하게 설명해주세요.(3점)

```
import torch
import torch.nn as nn

class DenseBlock(nn.Module):
	def __init__(self, in_dim, out_dim):
		super(DenseBlock, self).__init__()
		self.dense = nn.Linear(in_dim, out_dim)
		self.act = nn.ReLU()

	def forward(self, x):
		out = self.act(self.dense(x))
		return out
```


답) inplace=True는 입력 텐서를 직접 수정하여 메모리를 절약하고 약간의 속도 향상을 가져올 수 있다.
그러나 이는 입력 값의 손실 및 역전파 중 그래디언트 계산 문제를 초래할 수 있기에
주의 깊은 사용이 필요하며, 모델의 다른 부분이나 학습 과정에 영향을 미치지 않도록 신중하게 사용해야 한다.

### 1-7. Leaky ReLU 함수에 대한 코드입니다. 코드를 완성해주세요.(5점)

In [8]:
import torch
import torch.nn as nn

class DenseBlock(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(DenseBlock, self).__init__()
        self.dense = nn.Linear(in_dim, out_dim)  
        self.act = nn.ReLU()

    def forward(self, x):
        out = self.act(self.dense(x))
        return out


# 2. BTS맴버들의 이미지를 분류하는 코드를 완성해주세요.
- colab환경에서 우측 상단 화살표 -> 런타임 유형변경 -> 하드웨어 가속기 T4 GPU 선택 이후 실행해주세요.

- 제공된 bts_classification_simple파일을 구글드라이브에 업로드 한 후 아래코드를 실행해 주세요.

In [10]:
# 문제 x 그냥 실행해 주세요.
from __future__ import print_function
from torch import nn, optim, cuda
import torch
from torch.utils import data
from torchvision import datasets, transforms, models
import torch.nn.functional as F
import numpy as np
import time
import glob
from PIL import Image
import matplotlib.pyplot as plt

#from google.colab import drive
#drive.mount('/content/drive')

batch_size = 4
#device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training customized BTS Classfication Data on {device}')


Training customized BTS Classfication Data on cpu


### 2-1. 아래 코드를 완성하세요.(5점)
- 본인 구글드라이브의 알맞은 경로 입력

In [19]:
# Assuming `cls_dict` is defined as follows:
cls_dict = {0: 'BTS J-Hope', 1: 'BTS Jimin', 2: 'BTS Jin', 3: 'BTS Jungkook', 4: 'BTS RM', 5: 'BTS Suga', 6: 'BTS V'}

# Loading Training Data
all_image = glob.glob('./bts_classification_simple/train/*/*.jpg')
all_image.sort()
image_list = []
label_list = []

for item in all_image:
    image_list.append(np.array(Image.open(item)))
    label_folder = item.split('/')[-2]
    label_list.append(int(label_folder))

print("Loading Training Data... ")

all_image_tensor = torch.from_numpy((np.array(image_list).transpose(0, 3, 1, 2)).astype(np.float32))
target_tensor = torch.tensor(np.array(label_list))

train_dataset = torch.utils.data.TensorDataset(all_image_tensor, target_tensor)

# Loading Test Data
test_image = glob.glob('./bts_classification_simple/test/*/*.jpg')
test_image.sort()
test_image_list = []
test_label_list = []

for item in test_image:
    test_image_list.append(np.array(Image.open(item)))
    label_folder = item.split('/')[-2]
    test_label_list.append(int(label_folder))

print("Loading Test Data... ")

test_image_tensor = torch.from_numpy((np.array(test_image_list).transpose(0, 3, 1, 2)).astype(np.float32))
test_target_tensor = torch.tensor(np.array(test_label_list))

test_dataset = torch.utils.data.TensorDataset(test_image_tensor, test_target_tensor)

# Data loader
batch_size = 32  # Set appropriate batch size
train_loader = data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=1, shuffle=True)

ValueError: invalid literal for int() with base 10: 'bts_classification_simple'

### 2-2. 아래 코드를 완성하세요.(10점)
- 활성화함수 : relu
- 손실함수 : CrossEntropy
- optimizer : SGD
- learning rate = 0.001

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Conv2d(3, 16, 3, padding=1)
        self.l2 = nn.Conv2d(16, 32, 3, padding=1)
        self.l3 = nn.Conv2d(32, 32, 3, padding=1)
        self.l4 = nn.Conv2d(32, 32, 3, padding=1)
        self.l5 = nn.Conv2d(32, 32, 3, padding=1)
        self.l6 = nn.Linear(1568, 120)
        self.l7 = nn.Linear(120, 10)  # Assuming 10 classes for classification

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.l1(x)), 2)
        x = F.max_pool2d(F.relu(self.l2(x)), 2)
        x = F.max_pool2d(F.relu(self.l3(x)), 2)
        x = F.max_pool2d(F.relu(self.l4(x)), 2)
        x = F.max_pool2d(F.relu(self.l5(x)), 2)
        x = x.view(-1, 1568)
        x = F.relu(self.l6(x))
        return self.l7(x)

# Model Initialize
model = Net()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer Initialize
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)

### 2-3. 아래 코드를 완성해 주세요(10점)

In [15]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()  # optimizer의 기울기를 초기화합니다.
        output = model(data)

        loss = criterion(output, target)  # 손실을 계산합니다.
        loss.backward()  # 역전파 단계를 수행하여 기울기를 계산합니다.
        optimizer.step()  # 가중치를 업데이트합니다.

        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(img_show=False):
    model.eval()  # 모델을 평가 모드로 설정합니다.
    test_loss = 0
    correct = 0
    cnt = 0
    with torch.no_grad():  # 테스트 시에는 기울기를 계산하지 않습니다.
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            # 배치 손실을 더합니다.
            test_loss += criterion(output, target).item()

            # 예측값의 인덱스를 가져옵니다.
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

            if img_show and cnt < 5:  # 이미지를 보여줄 때
                plt.imshow(((data[0].to("cpu")).numpy() * 0.0039216).transpose(1, 2, 0))
                plt.show()
                print("Target: ", cls_dict[int(target.to("cpu").numpy())])
                print("Pred: ", cls_dict[int(pred.to("cpu").numpy())])

            cnt += 1

    test_loss /= len(test_loader.dataset)
    print(f'==================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)}'
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

In [16]:
# 학습후 매 Epoch당 test Accuracy를 확인해 보세요.
since = time.time()
for epoch in range(1, 11):
  epoch_start = time.time()
  train(epoch)
  m, s = divmod(time.time() - epoch_start, 60)
  print(f'Training time: {m:.0f}m {s:.0f}s')
  if epoch == 1:
    test(True)

  if epoch < 10:
    test()
  if epoch == 10:
    test(True)
  m, s = divmod(time.time() - epoch_start, 60)
  print(f'Tesing time: {m:.0f}m {s:.0f}s')

m, s = divmod(time.time() - since, 60)
print(f'Total time : {m:.0f}m {s: .0f}s \nModel was trained on {device}!')

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [4, 10]